In [31]:
import requests

res = requests.get("https://localhost:8000/openapi.json", verify=False)
print(res.json())

{'openapi': '3.1.0', 'info': {'title': 'FastAPI', 'version': '0.1.0'}, 'paths': {'/': {'get': {'summary': 'Welcome', 'operationId': 'welcome__get', 'responses': {'200': {'description': 'Successful Response', 'content': {'application/json': {'schema': {}}}}}}}, '/tickers': {'get': {'summary': 'Get Tickers', 'operationId': 'get_tickers_tickers_get', 'responses': {'200': {'description': 'Successful Response', 'content': {'application/json': {'schema': {}}}}}}}, '/returns/{ticker}/{start_date}/{end_date}': {'get': {'summary': 'Get Returns', 'operationId': 'get_returns_returns__ticker___start_date___end_date__get', 'parameters': [{'name': 'start_date', 'in': 'path', 'required': True, 'schema': {'type': 'string', 'title': 'Start Date'}}, {'name': 'end_date', 'in': 'path', 'required': True, 'schema': {'type': 'string', 'title': 'End Date'}}, {'name': 'ticker', 'in': 'path', 'required': True, 'schema': {'type': 'string', 'title': 'Ticker'}}], 'responses': {'200': {'description': 'Successful Re

/Users/richardgoldman/opt/anaconda3/envs/finapi/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [32]:
"""
Things we need to return:
1) endpoint
2) request method (GET, POST, PUT, etc.)
3)  a) for GET methods:
        - a dictionary of parameters
    b) for POST methods:
        - a dictionary representing the body of the request
"""
r = res.json()
output = []
for path, value in r['paths'].items():
    for method, info in value.items():
        item = {}
        item['endpoint'] = path
        item['method'] = method
        item['path_parameters'] = [{'name': p['name'], 'required': ['required']} for p in info.get('parameters',[]) if p['in'] == 'path']
        item['query_parameters'] = [{'name': p['name'], 'required': ['required']} for p in info.get('parameters',[]) if p['in'] == 'query']
        # Parse body parameters
        body_parameters = []
        if 'requestBody' in info:
            request_body = info['requestBody']
            if 'content' in request_body:
                for content_type, content_info in request_body['content'].items():
                    if 'schema' in content_info:
                        schema = content_info['schema']
                        if '$ref' in schema:
                            ref = schema['$ref']
                            # Resolve the reference if needed (this example assumes direct schema)
                            ref_name = ref.split('/')[-1]
                            body_parameters = [{'name': k, 'required': True} for k, v in r['components']['schemas'][ref_name]['properties'].items()]
                        elif 'properties' in schema:
                            body_parameters = [{'name': k, 'required': schema['required'] if 'required' in schema and k in schema['required'] else False} for k, v in schema['properties'].items()]

        item['body_parameters'] = body_parameters
        output.append(item)
    
print(output)

[{'endpoint': '/', 'method': 'get', 'path_parameters': [], 'query_parameters': [], 'body_parameters': []}, {'endpoint': '/tickers', 'method': 'get', 'path_parameters': [], 'query_parameters': [], 'body_parameters': []}, {'endpoint': '/returns/{ticker}/{start_date}/{end_date}', 'method': 'get', 'path_parameters': [{'name': 'start_date', 'required': ['required']}, {'name': 'end_date', 'required': ['required']}, {'name': 'ticker', 'required': ['required']}], 'query_parameters': [], 'body_parameters': []}, {'endpoint': '/correlation/{ticker1}/{ticker2}/{start_date}/{end_date}', 'method': 'get', 'path_parameters': [{'name': 'ticker1', 'required': ['required']}, {'name': 'ticker2', 'required': ['required']}, {'name': 'start_date', 'required': ['required']}, {'name': 'end_date', 'required': ['required']}], 'query_parameters': [], 'body_parameters': []}, {'endpoint': '/correlation_matrix/', 'method': 'post', 'path_parameters': [], 'query_parameters': [], 'body_parameters': [{'name': 'tickers',

In [22]:
r['paths'].keys()

dict_keys(['/', '/tickers', '/returns/{ticker}/{start_date}/{end_date}', '/correlation/{ticker1}/{ticker2}/{start_date}/{end_date}', '/correlation_matrix/', '/goldman/', '/functions.json'])